In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler # eliminate outliers
from sklearn import svm
import warnings                   # To ignore the warnings
warnings.filterwarnings("ignore")

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
valid = pd.read_csv('valid.csv')

# Find columns with missing values and count how many missing values in each column
missing_columns = train.columns[train.isnull().any()]
missing_counts = train[missing_columns].isnull().sum()

# Print the columns with missing values and their corresponding missing value counts
print("shape of train: ", train.shape)
for column in missing_columns:
    print(f"Column '{column}' has {missing_counts[column]} missing values.")

Label 2 contains missing values in the training dataset, but given that the dataset has almost 30,000 rows, removing 480 rows with missing values doesn't significantly impact the dataset's overall size or quality.

In [ ]:
L1 = "label_1" #Speaker ID
L2 = "label_2" #Speaker age
L3 = "label_3" #Speaker gender
L4 = "label_4" #Speaker accent
LABELS = [L1, L2, L3, L4,]
AGE_LABEL = L2
FEATURES = [f'feature_{i}' for i in range(1,769)]

In [ ]:
train_df = train.copy()
test_df = test.copy()
valid_df = valid.copy()

train_df.head()

In [ ]:
train_df[LABELS + [FEATURES[i] for i in range(0,267,32)]].describe()

In [ ]:
x_train = {}
x_valid = {}
x_test = {}

y_train = {}
y_valid = {}
y_test = {}

#create dictionaries for each label
for target_label in LABELS:
  tr_df = train_df[train_df['label_2'].notna()] if target_label == "label_2" else train_df
  vl_df = valid_df[valid_df['label_2'].notna()] if target_label == "label_2" else valid_df
  te_df = test_df

  scaler = RobustScaler()
  # x_train_features = tr_df.drop(LABELS, axis=1)

  x_train[target_label] = pd.DataFrame(scaler.fit_transform(tr_df.drop(LABELS, axis=1)), columns=FEATURES)
  y_train[target_label] = tr_df[target_label]

  x_valid[target_label] = pd.DataFrame(scaler.transform(vl_df.drop(LABELS, axis=1)), columns=FEATURES)
  y_valid  [target_label] = vl_df[target_label]

  x_test[target_label] = pd.DataFrame(scaler.transform(te_df.drop(["ID"],axis=1)), columns=FEATURES)
  # y_test[target_label] = te_df[target_label] <- need to predict




In the provided code, it removes rows with NaN values in Label 2. All four labels are then organized into a dictionary for easier handling and convenience.

# Support Functions

### Cross validation

In [ ]:
def crossValidation(model, x_train, y_train):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, x_train, y_train, cv=kf)

    mean_accuracy = scores.mean()
    std_accuracy = scores.std()
    # Print the cross-validation scores
    model_name = type(model).__name__
    print('--',model_name,'--')
    print("Cross-validation scores:", scores)
    print(f"Standard Deviation: {std_accuracy:.2f}")
    print("***Mean Accuracy***: {:.2f}%".format(mean_accuracy * 100))
    print('\n\n')


### Random Search for hyper parameter tuning 

In [ ]:
def randomSearch(modelName, x_train, y_train):

    param_dist_svm = {
        'C': [100,10,1],
        'kernel': ['rbf','linear','poly','sigmoid'],
        'gamma': ['scale','auto',1,10],
        'degree': [1,2,3],  # For the polynomial kernel
        'class_weight' : ['None','balanced']
    }

    param_dist_catBoost = {
        'depth': [2,6,10],
        'learning_rate': [0.1,1,10],
        'l2_leaf_reg': [1,2],
        'random_strength': [0,1],
    }

    param_dist_randomForrest = {
        'n_estimators': [1,10,100],
        'max_depth': [1,10],
        'min_samples_split': [1,10],
        'min_samples_leaf': [1,10],
    }

    svm = SVC()
    catBoost = CatBoostClassifier(iterations=100,task_type="GPU",devices='0:1')
    randomForrest = RandomForestClassifier()

    model = ''
    param_dist = ''
    nJobs = -1

    if(modelName=="svm"):
        model = svm
        param_dist = param_dist_svm
        nJobs = -1
    elif(modelName=="catBoost"):
        model = catBoost
        param_dist = param_dist_catBoost
        nJobs = 1
    elif(modelName=="randomForest"):
        model = randomForrest
        param_dist = param_dist_randomForrest
        nJobs = -1

    

    random_search = RandomizedSearchCV(
        model, param_distributions=param_dist, n_iter=10, cv=5, n_jobs=nJobs, random_state=42, scoring='accuracy'
    )

    random_search.fit(x_train, y_train)

    best_params = random_search.best_params_
    best_model = random_search.best_estimator_

    print("best parameters:", best_params)
    return best_params

### Grid Search for hyper parameter tuning 

In [ ]:
def gridSearch(modelName, param_grid, model, x_train, y_train):
    nJobs = -1
    if(modelName=="svm"):
        nJobs = -1
    elif(modelName=="catBoost"):
        nJobs = 1
    elif(modelName=="randomForest"):
        nJobs = -1

    # Perform grid search with cross-validation
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=nJobs, verbose=3)
    grid_search.fit(x_train, y_train)

    # Get the best hyperparameters
    best_params = grid_search.best_params_
    print("best parameters:", best_params)

In [ ]:
def generate_param_grid(best_params, modelName):
    param_grid = {}

    if modelName == 'svc' or 'svm' :
        param_grid = {
            'kernel': [best_params['kernel']],
            'gamma': ['scale', 'auto'] if best_params['gamma'] == 'scale' else [best_params['gamma']],
            'degree': [best_params['degree']],
            'class_weight': [best_params['class_weight']],
            'C': [ best_params['C'], 2*int(best_params['C']) ]
        }
    elif modelName == 'catBoost' :
        param_grid = {
            'depth': [ best_params['depth']-1, best_params['depth'], best_params['depth']+1 ],
            'learning_rate': [ best_params['learning_rate'], int(best_params['learning_rate'])*2 ],
            'l2_leaf_reg': [best_params['l2_leaf_reg']],
            'random_strength': [best_params['random_strength']],
        }
    elif modelName == 'randomForest' :
        param_grid = {
            'n_estimators': [best_params['n_estimators'], int(best_params['n_estimators'])*2],
            'max_depth': [best_params['max_depth'], int(best_params['max_depth'])*2],
            'min_samples_split': [best_params['min_samples_split']],
            'min_samples_leaf': [best_params['min_samples_leaf']],
        }
    
    return param_grid

### Create output csv

In [ ]:
def create_csv_from_labels(y_valid_pred, output_file):
    # Create a DataFrame with the label data

    data = {
        'ID': range(1, len(y_valid_pred[0]) + 1),
        'label_1': y_valid_pred[0],  # Assuming label_1 is the first column in y_valid_pred
        'label_2': y_valid_pred[1],  # Assuming label_2 is the second column in y_valid_pred
        'label_3': y_valid_pred[2],  # Assuming label_3 is the third column in y_valid_pred
        'label_4': y_valid_pred[3]   # Assuming label_4 is the fourth column in y_valid_pred
    }
    
    df = pd.DataFrame(data)
    
    # Save the DataFrame to a CSV file
    df.to_csv(output_file, index=False)

### Model Train

In [ ]:
def modelTrain( model, x_train, y_train, x_valid=None, y_valid=None, x_test=None):
    # Train
    model.fit(x_train, y_train)
    y_test_pred = False
    model_name = type(model).__name__

    if(not x_valid.empty):
        # Predict
        y_valid_pred = model.predict(x_valid)
        # Accuracy
        print(model_name,"accuracy_score for validation data set: ",metrics.accuracy_score(y_valid, y_valid_pred))

    if(not x_test.empty):
        ########### TEST ##############
        y_test_pred = model.predict(x_test)

    return y_test_pred

# Label training

In [ ]:
all_labels=[]
# This array is using to create the final output csv of all predicted labels.

# Label 1

In [ ]:
Label = L1
pca_NComponents = 0.97

In [ ]:
# Checking whether the dataset is bias or not
print(train_df[Label].unique())
print(train_df[Label].value_counts())
print(train_df.info())

According to the above information label 1 is not bias and its not compulsory to do over-sampling a under-sampling.

In [ ]:
x_train_df = x_train[Label].copy()
y_train_df = y_train[Label].copy()

x_valid_df = x_valid[Label].copy()
y_valid_df = y_valid[Label].copy()

x_test_df = x_test[Label].copy()
# y_test__df = y_test[L1].copy() <- need to predict

In [ ]:
# Feature extraction (dimension reduction)
pca = PCA(n_components=pca_NComponents, svd_solver='full')
pca.fit(x_train_df)
x_train_df_pca = pd.DataFrame(pca.transform(x_train_df)) #train
x_valid_df_pca = pd.DataFrame(pca.transform(x_valid_df)) #valid
x_test_df_pca = pd.DataFrame(pca.transform(x_test_df)) #test
print('Shape after PCA: ',x_train_df_pca.shape)

First of all, it is necessary to select the best classification algorithm for model training. Therefore, cross-validation techniques have been used to choose the best classifier. Four different classifiers have been applied for cross-validation.

In [ ]:
# Before feature engineering cross validation and checking accuracy

svm_classifier = svm.SVC()
knn_classifier = KNeighborsClassifier(n_neighbors=5)
catBoost_classifier = CatBoostClassifier(iterations=100,task_type="GPU")
randomForest_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

crossValidation(svm_classifier, x_train_df, y_train_df)
crossValidation(knn_classifier, x_train_df, y_train_df)
crossValidation(catBoost_classifier, x_train_df, y_train_df)
crossValidation(randomForest_classifier, x_train_df, y_train_df)


svm - 92.82% <br/>
knn - nan% <br/>
catBoost - 84.45% <br/>
random forrest - 84.96% <br/>
Therefore SVM is the best classifier to use further stuff.

In [ ]:
# After dimension reduction (with PCA) cross validation and checking accuracy
crossValidation(svm_classifier, x_train_df_pca, y_train_df)

Based on the previous observation, the accuracy has not significantly decreased even though PCA has been applied. Therefore, the dataset that underwent PCA is being used for further analysis.

In [ ]:
# Hyper parameter tuning with random search 
best_params_from_random_search = randomSearch('svm', x_train_df_pca, y_train_df)

Random search selects the best hyper-parameters for the chosen classifier randomly. Using these parameters, create a suitable parameter grid for grid search.

In [ ]:
# Hyper parameter tuning with grid search 
param_dist_svm = generate_param_grid(best_params_from_random_search, 'svm')

gridSearch('svm', param_dist_svm, svm_classifier, x_train_df_pca, y_train_df)

The combination of random search followed by grid search strikes a balance between exploration and exploitation of the hyperparameter space, making hyperparameter tuning more efficient and effective in finding good model configurations.

In [ ]:
svm_classifier = svm.SVC(kernel='rbf', C=1, class_weight='balanced')

#predict for the test dataset
y_test_pred = modelTrain(svm_classifier, x_train_df_pca, y_train_df, x_valid_df_pca, y_valid_df, x_test_df_pca)

all_labels.append(y_test_pred)


# Label 2

In [ ]:
Label = L2
pca_NComponents = 0.97

In [ ]:
# Checking whether the dataset is bias or not
print(train_df[Label].unique())
print(train_df[Label].value_counts())
print(train_df.info())

According to the above information label 2 is not bias and its not compulsory to do over-sampling a under-sampling.

In [ ]:
x_train_df = x_train[Label].copy()
y_train_df = y_train[Label].copy()

x_valid_df = x_valid[Label].copy()
y_valid_df = y_valid[Label].copy()

x_test_df = x_test[Label].copy()
# y_test__df = y_test[L1].copy() <- need to predict

In [ ]:
# dimension reduction
pca = PCA(n_components=pca_NComponents, svd_solver='full')
pca.fit(x_train_df)
x_train_df_pca = pd.DataFrame(pca.transform(x_train_df)) #train
x_valid_df_pca = pd.DataFrame(pca.transform(x_valid_df)) #valid
x_test_df_pca = pd.DataFrame(pca.transform(x_test_df)) #test
print('Shape after PCA: ',x_train_df_pca.shape)

First of all, it is necessary to select the best classification algorithm for model training. Therefore, cross-validation techniques have been used to choose the best classifier. Four different classifiers have been applied for cross-validation.

In [ ]:
# Before feature engineering cross validation and checking accuracy

svm_classifier = svm.SVC()
knn_classifier = KNeighborsClassifier(n_neighbors=5)
catBoost_classifier = CatBoostClassifier(iterations=100,task_type="GPU")
randomForest_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

crossValidation(svm_classifier, x_train_df, y_train_df)
crossValidation(knn_classifier, x_train_df, y_train_df)
crossValidation(catBoost_classifier, x_train_df, y_train_df)
crossValidation(randomForest_classifier, x_train_df, y_train_df)


svm - 92.82% <br/>
knn - nan% <br/>
catBoost - 84.45% <br/>
random forrest - 84.96% <br/>
Therefore SVM is the best classifier to use further stuff.

In [ ]:
# After dimension reduction (with PCA) cross validation and checking accuracy
crossValidation(svm_classifier, x_train_df_pca, y_train_df)

Based on the previous observation, the accuracy has not significantly decreased even though PCA has been applied. Therefore, the dataset that underwent PCA is being used for further analysis.

In [ ]:
# Hyper parameter tuning with random search 
best_params_from_random_search = randomSearch('svm', x_train_df_pca, y_train_df)

Random search selects the best hyper-parameters for the chosen classifier randomly. Using these parameters, create a suitable parameter grid for grid search.

In [ ]:
# Hyper parameter tuning with grid search 
param_dist_svm = generate_param_grid(best_params_from_random_search, 'svm')

gridSearch('svm', param_dist_svm, svm_classifier, x_train_df_pca, y_train_df)

The combination of random search followed by grid search strikes a balance between exploration and exploitation of the hyperparameter space, making hyperparameter tuning more efficient and effective in finding good model configurations.

In [ ]:
svm_classifier = svm.SVC(kernel='rbf', C=1, class_weight='balanced')

y_test_pred = modelTrain(svm_classifier, x_train_df_pca, y_train_df, x_valid_df_pca, y_valid_df, x_test_df_pca)

all_labels.append(y_test_pred)


# Label 3

In [ ]:
Label = L3
pca_NComponents = 0.97

In [ ]:
# Checking whether the dataset is bias or not
print(train_df[Label].unique())
print(train_df[Label].value_counts())
print(train_df.info())

According to the above information label 3 is bias to the value 1. Therefore, its necessary to do over-sampling and under-sampling.

In [ ]:
x_train_df = x_train[Label].copy()
y_train_df = y_train[Label].copy()

x_valid_df = x_valid[Label].copy()
y_valid_df = y_valid[Label].copy()

x_test_df = x_test[Label].copy()
# y_test__df = y_test[L1].copy() <- need to predict

In [ ]:
pca = PCA(n_components=pca_NComponents, svd_solver='full')
pca.fit(x_train_df)
x_train_df_pca = pd.DataFrame(pca.transform(x_train_df)) #train
x_valid_df_pca = pd.DataFrame(pca.transform(x_valid_df)) #valid
x_test_df_pca = pd.DataFrame(pca.transform(x_test_df)) #test
print('Shape after PCA: ',x_train_df_pca.shape)

First of all, it is necessary to select the best classification algorithm for model training. Therefore, cross-validation techniques have been used to choose the best classifier. Four different classifiers have been applied for cross-validation.

In [ ]:
# Before feature engineering cross validation and checking accuracy

svm_classifier = svm.SVC()
knn_classifier = KNeighborsClassifier(n_neighbors=5)
catBoost_classifier = CatBoostClassifier(iterations=100,task_type="GPU")
randomForest_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

crossValidation(svm_classifier, x_train_df, y_train_df)
crossValidation(knn_classifier, x_train_df, y_train_df)
crossValidation(catBoost_classifier, x_train_df, y_train_df)
crossValidation(randomForest_classifier, x_train_df, y_train_df)


svm - 92.82% <br/>
knn - nan% <br/>
catBoost - 84.45% <br/>
random forrest - 84.96% <br/>
Therefore SVM is the best classifier to use further stuff.

In [ ]:
# After dimension reduction (with PCA) cross validation and checking accuracy
crossValidation(svm_classifier, x_train_df_pca, y_train_df)

Based on the previous observation, the accuracy has not significantly decreased even though PCA has been applied. Therefore, the dataset that underwent PCA is being used for further analysis.

In [ ]:
# Hyper parameter tuning with random search 
best_params_from_random_search = randomSearch('svm', x_train_df_pca, y_train_df)

Random search selects the best hyper-parameters for the chosen classifier randomly. Using these parameters, create a suitable parameter grid for grid search.

In [ ]:
# Hyper parameter tuning with grid search 
param_dist_svm = generate_param_grid(best_params_from_random_search, 'svm')

gridSearch('svm', param_dist_svm, svm_classifier, x_train_df_pca, y_train_df)

The combination of random search followed by grid search strikes a balance between exploration and exploitation of the hyperparameter space, making hyperparameter tuning more efficient and effective in finding good model configurations.

In [ ]:
svm_classifier = svm.SVC(kernel='rbf', C=100, class_weight='balanced')

y_test_pred = modelTrain(svm_classifier, x_train_df_pca, y_train_df, x_valid_df_pca, y_valid_df, x_test_df_pca)

all_labels.append(y_test_pred)


# Label 4

In [ ]:
Label = L4
pca_NComponents = 0.97

In [ ]:
# Checking whether the dataset is bias or not
print(train_df[Label].unique())
print(train_df[Label].value_counts())
print(train_df.info())

According to the above information label 4 is bias to value 2 and 6. Therefore, its necessary to do over-sampling and under-sampling.

In [ ]:
x_train_df = x_train[Label].copy()
y_train_df = y_train[Label].copy()

x_valid_df = x_valid[Label].copy()
y_valid_df = y_valid[Label].copy()

x_test_df = x_test[Label].copy()
# y_test__df = y_test[L1].copy() <- need to predict

In [ ]:
pca = PCA(n_components=pca_NComponents, svd_solver='full')
pca.fit(x_train_df)
x_train_df_pca = pd.DataFrame(pca.transform(x_train_df)) #train
x_valid_df_pca = pd.DataFrame(pca.transform(x_valid_df)) #valid
x_test_df_pca = pd.DataFrame(pca.transform(x_test_df)) #test
print('Shape after PCA: ',x_train_df_pca.shape)

First of all, it is necessary to select the best classification algorithm for model training. Therefore, cross-validation techniques have been used to choose the best classifier. Four different classifiers have been applied for cross-validation.

In [ ]:
# Before feature engineering cross validation and checking accuracy

svm_classifier = svm.SVC()
knn_classifier = KNeighborsClassifier(n_neighbors=5)
catBoost_classifier = CatBoostClassifier(iterations=100,task_type="GPU")
randomForest_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

crossValidation(svm_classifier, x_train_df, y_train_df)
crossValidation(knn_classifier, x_train_df, y_train_df)
crossValidation(catBoost_classifier, x_train_df, y_train_df)
crossValidation(randomForest_classifier, x_train_df, y_train_df)


svm - 92.82% <br/>
knn - nan% <br/>
catBoost - 84.45% <br/>
random forrest - 84.96% <br/>
Therefore SVM is the best classifier to use further stuff.

In [ ]:
# After dimension reduction (with PCA) cross validation and checking accuracy
crossValidation(svm_classifier, x_train_df_pca, y_train_df)

Based on the previous observation, the accuracy has not significantly decreased even though PCA has been applied. Therefore, the dataset that underwent PCA is being used for further analysis.

In [ ]:
# Hyper parameter tuning with random search 
best_params_from_random_search = randomSearch('svm', x_train_df_pca, y_train_df)

Random search selects the best hyper-parameters for the chosen classifier randomly. Using these parameters, create a suitable parameter grid for grid search.

In [ ]:
# Hyper parameter tuning with grid search 
param_dist_svm = generate_param_grid(best_params_from_random_search, 'svm')
print(param_dist_svm)
# gridSearch('svm', param_dist_svm, svm_classifier, x_train_df_pca, y_train_df)

The combination of random search followed by grid search strikes a balance between exploration and exploitation of the hyperparameter space, making hyperparameter tuning more efficient and effective in finding good model configurations.

In [ ]:
svm_classifier = svm.SVC(kernel='poly', C=100, gamma='scale', degree=3, class_weight='balanced')

y_test_pred = modelTrain(svm_classifier, x_train_df_pca, y_train_df, x_valid_df_pca, y_valid_df, x_test_df_pca)

all_labels.append(y_test_pred)


# make output csv

In [ ]:
create_csv_from_labels(all_labels, 'layer7_output.csv')